In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 19.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 2.8 MB/s eta 0:00:00


In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="z398Krfh8O5e0AvgLKNK")
project = rf.workspace("aysha-altaf").project("stenosis-detection-89nbf")
version = project.version(4)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to stenosis-detection-4 in yolov8:: 100%|██████████| 31630/31630 [00:04<00:00, 7758.11it/s] 


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
from ultralytics import YOLO

In [7]:
import os

# Define the paths to the train, test, and valid directories
train_dir = '/kaggle/working/stenosis-detection-4/train'
test_dir = '/kaggle/working/stenosis-detection-4/test'
valid_dir = '/kaggle/working/stenosis-detection-4/valid'

# Function to count images in a directory
def count_images(directory):
    return sum([len(files) for r, d, files in os.walk(directory)])

# Count images in each directory
train_images = count_images(train_dir)
test_images = count_images(test_dir)
valid_images = count_images(valid_dir)

# Print the results
print(f"Total images in Train set: {train_images}")
print(f"Total images in Test set: {test_images}")
print(f"Total images in Valid set: {valid_images}")

Total images in Train set: 22472
Total images in Test set: 4570
Total images in Valid set: 4576


In [8]:
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12345)

In [9]:
model = YOLO("yolov8n")

100%|██████████| 6.25M/6.25M [00:00<00:00, 97.5MB/s]


In [10]:
import yaml


# Load the YAML file
yaml_file_path = '/kaggle/working/stenosis-detection-4/data.yaml'  # Example YAML file path

# Define the paths to your dataset directories
train_dir = '/kaggle/working/stenosis-detection-4/train'
test_dir = '/kaggle/working/stenosis-detection-4/test'
valid_dir = '/kaggle/working/stenosis-detection-4/valid'

# Read the YAML file
with open(yaml_file_path, 'r') as file:
    yaml_content = yaml.safe_load(file)

# Update the paths in the YAML file (ensure the keys match your YAML file's structure)
yaml_content['train'] = train_dir
yaml_content['test'] = test_dir
yaml_content['val'] = valid_dir  # If you have validation images

# Write the updated paths back to the YAML file
with open(yaml_file_path, 'w') as file:
    yaml.dump(yaml_content, file)

print(f"Updated YAML file paths for train, test, and valid images.")

Updated YAML file paths for train, test, and valid images.


In [11]:
results = model.train(
    data="/kaggle/working/stenosis-detection-4/data.yaml",  # Path to your dataset config
    epochs=60,  # Increase epochs for deeper training
    project='FYP_DL',  # Updated project name
    name='stenosis_detection',  # Updated run name
    save_period=5,  # Save checkpoints every 5 epochs
    batch=16,  # Larger batch size for faster training (adjust based on GPU memory)
    patience=5,  # Early stopping if no improvement for 10 epochs
    optimizer='Adam',  # Use Adam optimizer for better convergenc
    augment=True  # Apply augmentations for better generalization

)

Ultralytics 8.3.32 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/stenosis-detection-4/data.yaml, epochs=60, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=FYP_DL, name=stenosis_detection, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

100%|██████████| 755k/755k [00:00<00:00, 19.3MB/s]
2024-11-18 16:05:22,627	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-18 16:05:23,168	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 78.0MB/s]


AMP: checks passed ✅


...

      60/60      2.37G      1.742     0.8792      1.292          4        640: 100%|██████████| 703/703 [02:17<00:00,  5.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [00:12<00:00,  5.54it/s]

                   all       2288       2288      0.899      0.806      0.876      0.381



60 epochs completed in 2.543 hours.
Optimizer stripped from FYP_DL/stenosis_detection/weights/last.pt, 6.3MB
Optimizer stripped from FYP_DL/stenosis_detection/weights/best.pt, 6.3MB

Validating FYP_DL/stenosis_detection/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [00:22<00:00,  3.17it/s]


                   all       2288       2288      0.857      0.797      0.868      0.394
Speed: 0.3ms preprocess, 5.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to FYP_DL/stenosis_detection


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.2/887.2 kB 13.3 MB/s eta 0:00:00a 0:00:01


In [4]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
model = YOLO("/kaggle/input/test-dataset/best (2).pt")